In [9]:
import streamlit as st
import pandas as pd
import numpy as np
# Scikit-Learn models, metrics, preprocessing
from sklearn.impute import SimpleImputer

#Filter and ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Load and understand raw_data
df = pd.read_csv("C:/Users/sisharan/OneDrive - Microsoft/Documents/Maven_course/Uber_ETA_prediction/data/raw_data.csv")

# Top 5 rows
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Ordered,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,Multiple_deliveries,Festival,City_type,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [12]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Ordered',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'Multiple_deliveries', 'Festival', 'City_type', 'Time_taken(min)'],
      dtype='object')

In [13]:
# Dimensions of the dataframe
print("Train Dataset :", df.shape)

Train Dataset : (45593, 20)


In [14]:
# Finding datatype of each column in dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Ordered                 45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [15]:
#Check statistical values for fields with numerical datatype
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Restaurant_latitude,45593.0,17.017729,8.185109,-30.905562,12.933284,18.546947,22.728163,30.914057
Restaurant_longitude,45593.0,70.231332,22.883647,-88.366217,73.170000,75.898497,78.044095,88.433452
Delivery_location_latitude,45593.0,17.465186,7.335122,0.010000,12.988453,18.633934,22.785049,31.054057
Delivery_location_longitude,45593.0,70.845702,21.118812,0.010000,73.280000,76.002574,78.107044,88.563452
Vehicle_condition,45593.0,1.023359,0.839065,0.000000,0.000000,1.000000,2.000000,3.000000


In [16]:
#Check statistical values for fields with other than numerical datatype
df.describe(exclude=np.number).T

,count,unique,top,freq
ID,45593,45593,0x4607,1
Delivery_person_ID,45593,1320,PUNERES01DEL01,67
Delivery_person_Age,45593,23,35,2262
Delivery_person_Ratings,45593,29,4.8,7148
Order_Date,45593,44,15-03-2022,1192
Time_Ordered,45593,177,NaN,1731
Time_Order_picked,45593,193,21:30:00,496
Weatherconditions,45593,7,conditions Fog,7654
Road_traffic_density,45593,5,Low,15477
Type_of_order,45593,4,Snack,11533


Observations:

There are null values present in Time_Ordered column. String "Nan" value.
Cannot directly do impute as it is "hh:mm:ss" format.
Difference of time_ordered and order_picked gives food_preparation_time.
Median of the food_preparation_time used as snacks require less time to make, other foods like buffet and meals takes more time.
Data Formatting will be required for Weatherconditions & Time_taken(min) column.
Both numerical & categorical features are present.
ID & Delivery_person_ID will not be used to build the model.

In [17]:
# Update Column Names
def update_column_name(df):
    # Renaming Weatherconditions column
    df.rename(columns={'Weatherconditions': 'Weather_conditions'},inplace=True)

update_column_name(df)
print(df.columns)

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Ordered',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'Multiple_deliveries', 'Festival', 'City_type', 'Time_taken(min)'],
      dtype='object')


In [19]:
def extract_column_value(df):
    '''
    Cleans and extracts:
    - Numerical value from "Time_taken(min)"
    - Weather description from "Weather_conditions"
    - City code prefix from "Delivery_person_ID"
    '''

    # Extract numeric part from 'Time_taken(min)'
    def clean_time(x):
        try:
            # Handles formats like '30 (min)' or '30min' or just '30'
            x = str(x)
            # Extract all digits
            return int(''.join(filter(str.isdigit, x)))
        except:
            return None

    df['Time_taken(min)'] = df['Time_taken(min)'].apply(clean_time)

    # Extract weather description safely
    df['Weather_conditions'] = df['Weather_conditions'].apply(
        lambda x: str(x).split()[1] if pd.notnull(x) and len(str(x).split()) > 1 else None
    )

    # Extract city code from Delivery_person_ID
    df['City_code'] = df['Delivery_person_ID'].apply(
        lambda x: str(x).split("RES")[0] if pd.notnull(x) else None
    )



In [20]:
extract_column_value(df)
df[['Time_taken(min)','Weather_conditions','City_code']].head()

,Time_taken(min),Weather_conditions,City_code
0,24,Sunny,INDO
1,33,Stormy,BANG
2,26,Sandstorms,BANG
3,21,Sunny,COIMB
4,30,Cloudy,CHEN


In [21]:
# Unique values of some categorical columns of df_train
columns = ['Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
      'Multiple_deliveries', 'Festival', 'City_type']

for column in columns:
    unique_values = df[column].unique().tolist()
    print(column, ":", unique_values)

Weather_conditions : ['Sunny', 'Stormy', 'Sandstorms', 'Cloudy', 'Fog', 'Windy', 'NaN']
Road_traffic_density : ['High ', 'Jam ', 'Low ', 'Medium ', 'NaN ']
Vehicle_condition : [2, 0, 1, 3]
Type_of_order : ['Snack ', 'Drinks ', 'Buffet ', 'Meal ']
Type_of_vehicle : ['motorcycle ', 'scooter ', 'electric_scooter ', 'bicycle ']
Multiple_deliveries : ['0', '1', '3', 'NaN ', '2']
Festival : ['No ', 'Yes ', 'NaN ']
City_type : ['Urban ', 'Metropolitian ', 'Semi-Urban ', 'NaN ']


In [22]:
# Drop Columns which won't be use for building model
def drop_columns(df):
    df.drop(['ID','Delivery_person_ID'],axis=1,inplace=True)

print("Before No. of columns: ",df.shape[1])
drop_columns(df)
print("After No. of columns: ",df.shape[1])

#Check for Duplicate Values
if (len(df[df.duplicated()])>0):
    print("\nThere are Duplicate values present")
else:
    print("\nThere is no duplicate value present")

Before No. of columns:  21
After No. of columns:  19

There is no duplicate value present


In [23]:
# Update datatypes
def update_datatype(df):
    df['Delivery_person_Age'] = df['Delivery_person_Age'].astype('float64')
    df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype('float64')
    df['Multiple_deliveries'] = df['Multiple_deliveries'].astype('float64')
    df['Order_Date']=pd.to_datetime(df['Order_Date'],format="%d-%m-%Y")

update_datatype(df)

In [24]:
# Convert String 'NaN' to np.nan
'''
It seems "NaN" strings have a space after them, convert both "NaN " and "NaN" to object Null values
So, we use regex approach.
'''

def convert_nan(df):
    df.replace('NaN', float(np.nan), regex=True,inplace=True)

convert_nan(df)

In [25]:
# Check null values
df.isnull().sum().sort_values(ascending=False)

Delivery_person_Ratings        1908
Delivery_person_Age            1854
Time_Ordered                   1731
City_type                      1200
Multiple_deliveries             993
Weather_conditions              616
Road_traffic_density            601
Festival                        228
Order_Date                        0
Delivery_location_longitude       0
Time_Order_picked                 0
Delivery_location_latitude        0
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
Restaurant_longitude              0
Restaurant_latitude               0
Time_taken(min)                   0
City_code                         0
dtype: int64

In [26]:
# Handle null values
def handle_null_values(df):
    df['Delivery_person_Age'].fillna(np.random.choice(df['Delivery_person_Age']), inplace=True)    # Names chosen randomly
    df['Weather_conditions'].fillna(np.random.choice(df['Weather_conditions']), inplace=True)      # weather chosen randomly

    mode_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')                      # Filled with the most common value
    mode_cols = ["Road_traffic_density",
             "Multiple_deliveries", "Festival", "City_type"]

    for col in mode_cols:
        df[col] = mode_imp.fit_transform(df[col].to_numpy().reshape(-1,1)).ravel()

    # Mean is affected by extreme values, mode gives highest frequency which is inappropriate
    # So, median is appropriate measure for imputing
    df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median(), inplace=True)

    df["Time_Ordered"] = df["Time_Ordered"].fillna(df["Time_Order_picked"])

handle_null_values(df)
df.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Ordered                   0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
Multiple_deliveries            0
Festival                       0
City_type                      0
Time_taken(min)                0
City_code                      0
dtype: int64

In [28]:
df.to_csv("C:/Users/sisharan/OneDrive - Microsoft/Documents/Maven_course/Uber_ETA_prediction/data/data_cleaned.csv")